<a href="https://colab.research.google.com/github/degru82/programmers-de/blob/master/dataengineering_w2_mau.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로그래머스 데이터엔지니어링 2주차

1. db-conn 함수의 작성
1. MAU 계산을 SQL을 사용해서
1. MAU 계산을 Pandas를 사용해서

----
## 1. 먼저 db-conn 함수를 작성한다

In [24]:
import psycopg2

def connect_to_redshift(
    user: str, pwd: str,
    host: str="grepp-data.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com",
    port: int=5439, dbname: str='dev',
)->tuple:
    """
    레드시프트 데이터베이스에 연결하고 정보를 리턴합니다

    returns (connection, cursor)

    inputs
    - user: username
    - pwd: password
    - host: redshift url (default: grepp-data.blahblah)
    - port: port number (default: 5439)
    - dbname: database name (default: dev)
    """

    info = \
        f'dbname={dbname} user={user} host={host} password={pwd} port={port}'
    conn = psycopg2.connect(info)

    return conn, conn.cursor()


connect_to_redshift?

In [4]:
# 실행환경이 콘솔이었다면, 환경변수에 지정했겠지만,
# 주피터 노트북에서 실행할 예정이므로 ID/PWD를 입력받습니다

user = input('Type Username: ')
pwd = input('Type Password: ')

conn, cursor = connect_to_redshift(user, pwd)
conn, cursor

Type Username: degru82
Type Password: Degru821!


(<connection object at 0x7f4b457f9898; dsn: 'user=degru82 password=xxx dbname=dev host=grepp-data.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com port=5439', closed: 0>,
 <cursor object at 0x7f4b45805be8; closed: 0>)

----
## 2. MAU 계산을 SQL을 사용해서

In [5]:
query = '''
SELECT 
    COUNT(DISTINCT USC.userid)
    , EXTRACT(YEAR FROM ST.ts) AS YYYY
    , EXTRACT(MONTH FROM ST.ts) AS MM
FROM raw_data.user_session_channel USC

JOIN raw_data.session_timestamp ST
ON USC.sessionid=ST.sessionid

GROUP BY YYYY, MM
'''

cursor.execute(query)
result = cursor.fetchall()

In [7]:
for num_users, year, month in result:

    print(f'{num_users} MAUs on {year}-{month:02}')

281 MAUs on 2019-05
459 MAUs on 2019-06
639 MAUs on 2019-09
763 MAUs on 2019-10
623 MAUs on 2019-07
662 MAUs on 2019-08
721 MAUs on 2019-11


----
## 3. MAU 계산을 Pandas를 사용해서
- 두 개의 테이블을 읽어와서
- 합쳐서 하나의 테이블을 만들고 (join)
- 연도와 월 정보를 추출해서 새로운 컬럼을 만든 뒤,
- 해당 연월에 유니크한 사용자 아이디 개수를 구한다

In [8]:
import pandas.io.sql as sqlio

query = "select * from raw_data.user_session_channel"
df_usc = sqlio.read_sql(query, conn)
df_usc.head()



,userid,sessionid,channel
0,779,7cdace91c487558e27ce54df7cdb299c,Instagram
1,230,94f192dee566b018e0acf31e1f99a2d9,Naver
2,369,7ed2d3454c5eea71148b11d0c25104ff,Youtube
3,248,f1daf122cde863010844459363cd31db,Naver
4,676,fd0efcca272f704a760c3b61dcc70fd0,Instagram


In [10]:
query = "select * from raw_data.session_timestamp"
df_st = sqlio.read_sql(query, conn)
df_st.head()

,sessionid,ts
0,7cdace91c487558e27ce54df7cdb299c,2019-05-01 00:13:11.783
1,94f192dee566b018e0acf31e1f99a2d9,2019-05-01 00:49:46.073
2,7ed2d3454c5eea71148b11d0c25104ff,2019-05-01 10:18:43.210
3,f1daf122cde863010844459363cd31db,2019-05-01 13:10:56.413
4,fd0efcca272f704a760c3b61dcc70fd0,2019-05-01 13:45:19.793


In [15]:
import pandas as pd

df_merged = pd.merge(df_usc, df_st, how='outer', on='sessionid')
df_merged.tail()

,userid,sessionid,channel,ts
101515,45,691f008d42e361433415db5a7773d697,Instagram,2019-11-30 22:54:15
101516,2181,60d1f51c17418bb12b9bf94d289c7204,Google,2019-11-30 23:25:11
101517,2337,9cfaefd1e81f637fad6330ff16eb1f39,Organic,2019-11-30 23:46:39
101518,1285,f06a4ccd032e58412b6452b2536ecbf2,Youtube,2019-11-29 15:10:30
101519,2271,f551359ab7f6d759eb1880f554e5e815,Naver,2019-11-29 15:06:44


In [30]:
yyyymm = df_merged['ts'].apply(lambda x: x.strftime('%Y%m'))
df_merged['yyyymm'] = yyyymm
df_merged.head()

,userid,sessionid,channel,ts,yyyymm
0,779,7cdace91c487558e27ce54df7cdb299c,Instagram,2019-05-01 00:13:11.783,201905
1,230,94f192dee566b018e0acf31e1f99a2d9,Naver,2019-05-01 00:49:46.073,201905
2,369,7ed2d3454c5eea71148b11d0c25104ff,Youtube,2019-05-01 10:18:43.210,201905
3,248,f1daf122cde863010844459363cd31db,Naver,2019-05-01 13:10:56.413,201905
4,676,fd0efcca272f704a760c3b61dcc70fd0,Instagram,2019-05-01 13:45:19.793,201905


In [31]:
yyyymm.unique()

array(['201905', '201906', '201907', '201908', '201909', '201910',
       '201911'], dtype=object)

In [32]:
mau_yyyymm = {}

for yyyymm in yyyymm.unique():
    on_yyyymm = (df_merged['yyyymm'] == yyyymm)
    num_users = len(df_merged[on_yyyymm]['userid'].unique())
    mau_yyyymm[yyyymm] = num_users

mau_yyyymm

{'201905': 281,
 '201906': 459,
 '201907': 623,
 '201908': 662,
 '201909': 639,
 '201910': 763,
 '201911': 721}